In [1]:
import torch.nn as nn
import torch
import sys
sys.path.append("..")
from src.model import Laplace_fast

In [2]:
input = torch.randn(1, 2560)

WKN = Laplace_fast(32,64)
WKN_input = WKN(input)
WKN_input.shape

[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.
/Users/yentsokuo/git_repo/WKN_SSO/try/../src/model.py:41: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/Convolution.cpp:1009.)
  return F.conv1d(waveforms, self.filters, stride=1, padding='same', dilation=1, bias=None, groups=1)


torch.Size([32, 2560])

In [9]:
t = input.squeeze()
t.shape

torch.Size([32, 2560])

In [44]:
input = torch.randn(1, 2560)

c0 = nn.Conv1d(1, 32, kernel_size=32, padding='same')
z = c0(input)

z.shape

torch.Size([32, 2560])

In [45]:
# pool of size=3, stride=2
m1 = nn.MaxPool1d(2, stride=2)
a = m1(z)

a.shape

torch.Size([32, 1280])

In [46]:
c1 = nn.Conv1d(32, 16, kernel_size=32, padding='same')
b = c1(a)

b.shape

torch.Size([16, 1280])

In [47]:
m2 = nn.MaxPool1d(2, stride=2)
c = m1(b)

c.shape

torch.Size([16, 640])

In [48]:
c2 = nn.Conv1d(16, 32, kernel_size=32, padding='same')
d = c2(c)

d.shape

torch.Size([32, 640])

In [49]:
m3 = nn.MaxPool1d(2, stride=2)
e = m3(d)

e.shape


torch.Size([32, 320])

In [50]:
e = e.view(320, 32)
BiGRU = nn.GRU(input_size=32, hidden_size=8, num_layers=1, bidirectional=True)
output, hidden = BiGRU(e)

# 訪問最後一個時間步的隱藏狀態的形狀
print(output.shape)
# 或者訪問整個序列的隱藏狀態的形狀
print(hidden.shape)


torch.Size([320, 16])
torch.Size([2, 8])


In [51]:
f1 = nn.Flatten(0)
o1 = f1(output)

o1.shape

torch.Size([5120])

In [52]:
l1 = nn.Linear(5120, 64)
o2 = l1(o1)

o2.shape

torch.Size([64])

In [53]:
l2 = nn.Linear(64, 1)
o3 = l2(o2)

o3.shape

torch.Size([1])

In [66]:
di = torch.randn(32,1)
di_squeezed = di.squeeze()

print(di)

tensor([[-7.8523e-01],
        [ 4.5005e-01],
        [ 1.5834e-01],
        [-9.8807e-01],
        [-2.0539e-01],
        [ 1.0252e+00],
        [-3.6740e-01],
        [-1.2244e+00],
        [-2.3265e-02],
        [-1.5839e+00],
        [ 6.4034e-01],
        [-1.9665e+00],
        [ 1.7263e+00],
        [-3.7292e-01],
        [ 1.6391e+00],
        [-3.6448e-01],
        [-5.6928e-02],
        [ 3.9751e-01],
        [-8.9022e-02],
        [-1.2072e+00],
        [-1.5513e+00],
        [ 1.7884e+00],
        [ 8.3717e-01],
        [ 2.5921e-01],
        [ 3.7565e-01],
        [ 4.5042e-01],
        [ 1.2479e-01],
        [-3.2192e-01],
        [-1.5225e+00],
        [-5.8179e-01],
        [ 4.1697e-01],
        [-2.8003e-04]])


In [67]:
print(di_squeezed)

tensor([-7.8523e-01,  4.5005e-01,  1.5834e-01, -9.8807e-01, -2.0539e-01,
         1.0252e+00, -3.6740e-01, -1.2244e+00, -2.3265e-02, -1.5839e+00,
         6.4034e-01, -1.9665e+00,  1.7263e+00, -3.7292e-01,  1.6391e+00,
        -3.6448e-01, -5.6928e-02,  3.9751e-01, -8.9022e-02, -1.2072e+00,
        -1.5513e+00,  1.7884e+00,  8.3717e-01,  2.5921e-01,  3.7565e-01,
         4.5042e-01,  1.2479e-01, -3.2192e-01, -1.5225e+00, -5.8179e-01,
         4.1697e-01, -2.8003e-04])


In [65]:
x = torch.randn(17,32,320)
x_permuted = x.permute(0, 2, 1)

x_permuted.shape

torch.Size([17, 320, 32])

In [74]:
input.shape

torch.Size([1, 2560])

In [14]:
import pandas as pd
import numpy as np
import sys
sys.path.append("..")

import torch
from src.utils import *
from src.model import LA_WKN_BiGRU

pth_path = '/Users/yentsokuo/git_repo/WKN_SSO/src/your_model.pth'
# 加载已经训练好的模型
model = LA_WKN_BiGRU()  # 请替换为你的模型类
model.load_state_dict(torch.load(pth_path))
model.eval()  # 切换模型为评估模式

# 准备输入数据（震动数据），这里使用示例数据，你需要根据你的数据结构进行调整
input_path = '/Users/yentsokuo/git_repo/WKN_SSO/viberation_dataset/Test_set/Bearing1_3/acc_00001.csv'  # 一个(1, 2560)尺寸的震动数据
input_data = pd.read_csv(input_path, header=None, names=['hour', 'minute', 'second', 'microsecond', 'horiz accel', 'vert accel'])
inputs = input_data['horiz accel'].values.astype(float)
inputs = min_max_scale(inputs).reshape(1, -1)
inputs_tensor = torch.from_numpy(inputs.astype(np.float32))
# input_tensor = torch.Tensor(input_data)

In [15]:
inputs_tensor.shape

torch.Size([1, 2560])

In [12]:
#使用模型进行预测
with torch.no_grad():
    predicted_remaining_life = model(inputs_tensor)

# 处理预测结果，这取决于你的任务
# 在这个例子中，predicted_remaining_life是一个float，表示预测的剩余寿命
print(f'Predicted Remaining Life: {predicted_remaining_life.item()}')

torch.Size([2560])


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 2)